In [1]:
import numpy as np
import matplotlib.pyplot as plt
import awkward as ak
import mplhep as hep
import pandas as pd
hep.style.use(hep.style.CMS)
import hist as hist2

from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.nanoevents.methods import candidate, vector
from coffea.analysis_tools import Weights, PackedSelection

# we suppress ROOT warnings where our input ROOT tree has duplicate branches - these are handled correctly.
import warnings
import uproot
from coffea import processor
import warnings

warnings.filterwarnings('ignore')

In [ ]:
class triggerEffWbProcessor(processor.ProcessorABC):
    def __init__(self):
        self.make_output = lambda: {
            'pt_m_tr': hist2.Hist(
                hist2.axis.Regular(20, 30, 500, name='pt_m_tr'),
                ),
            'pt_m_tc': hist2.Hist(
                hist2.axis.Regular(20, 30, 500, name='pt_m_tc'),
                ),
            'eta_m_tr': hist2.Hist(
                hist2.axis.Regular(20, -2.4, 2.4, name='eta_m_tr'),
                ),
            'eta_m_tc': hist2.Hist(
                hist2.axis.Regular(20,-2.4, 2.4, name='eta_m_tc'),
                ),
            'met_tr': hist2.Hist(
                hist2.axis.Regular(20, 50, 500, name='met_tr'),
                ),
            'met_tc': hist2.Hist(
                hist2.axis.Regular(20, 50, 500, name='met_tc'),
                ),
            'pt_bjet_tr': hist2.Hist(
                hist2.axis.Regular(20, 30, 500, name='pt_bjet_tr'),
                ),
            'pt_bjet_tc': hist2.Hist(
                hist2.axis.Regular(20, 30, 500, name='pt_bjet_tc'),
                ),
            'eta_bjet_tr': hist2.Hist(
                hist2.axis.Regular(30, -2.1, 2.1, name='eta_bjet_tr'),
                ),
            'eta_bjet_tc': hist2.Hist(
                hist2.axis.Regular(30, -2.1, 2.1, name='eta_bjet_tc'),
                )
        }
    @property
    def accumulator(self):
        return self._accumulator

    def process(self, events):
        dataset = events.metadata['dataset']
        #weights = Weights(len(events), storeIndividual=True)
        #weights.add('genweight', events.genWeight)
        
        selection = PackedSelection()
        
        
        
        output =self.make_output()
        output['sum'] = len(events)
        b_jet=events.Jet[(events.Jet.jetId==6) & (events.Jet.puId==7) & (events.Jet.btagDeepFlavB>0.4506)]
        events["bjet"]=b_jet
        
        #trigger
        
        trigger_de_ref=events.HLT.Ele35_WPTight_Gsf
        trigger_central=events.HLT.IsoMu27
        
        #events["heepmuon"]=events.Muon[events.Muon.mediumId]
        #events["heepmuon"]=ak.sum(events.Muon.highPtId>0, axis=1)==2
        # Identifier Muon electron
        events["heepmuon"]=events.Muon[events.Muon.tightId]
        events["heeplecton"]=events.Electron[events.Electron.cutBased_HEEP]
        #Muon
        #& (ak.sum(events.Muon.highPtId>0, axis=1)==2)
        goodmuon = ((ak.sum(events.heepmuon.pt>0, axis=1)==1)
                &((ak.pad_none(events.heepmuon.pt,1)[:,0]>30))
                & ((abs(ak.pad_none(events.heepmuon.eta,2)[:,0])<2.4))
                )
       
        
        
        
                       
        #electron & (ak.sum(events.Electron.cutBased>0, axis=1)==2)
        goodelectron = ((ak.sum(events.heeplecton.pt>0, axis=1)==1)
                        &((ak.pad_none(events.heeplecton.pt, 1)[:,0]>30))
                        &(((0<abs(ak.firsts(events.heeplecton.eta))) & (abs(ak.firsts(events.heeplecton.eta))<2.4))
                        & ((1.57<abs(ak.firsts(events.heeplecton.eta))) | (abs(ak.firsts(events.heeplecton.eta))<1.4)))
                #&((ak.pad_none(events.Electron.charge, 2)[:,0])!=(ak.pad_none(events.Electron.charge, 2)[:,1]))
                )
        
        #b-jet
        
        goodbJets = ((ak.sum(events.bjet.pt>0, axis=1)>=1)
            &((ak.firsts(events.bjet.pt)>30)
            &(abs(ak.firsts(events.bjet.eta))<2.1)
             ))
        
        #met
        
        #goodMet=events.MET.pt>50
        
        #seleccion
        
        selection.add("trigger_de_ref",trigger_de_ref)
        selection.add("trigger_central",trigger_central)
        #selection.add("trigger_central_muon", trigger_central_muon)
        selection.add("muon",goodmuon)
        selection.add("bJet",goodbJets)
        selection.add("electron",goodelectron)
        #selection.add("Met",goodMet)
        
        #regions
        
        
        regions={"signal_trigger":["trigger_de_ref"],
         "signal_trigger_e":["trigger_de_ref","muon"],
         "signal_trigger_e_bjet":["trigger_de_ref","muon","bJet"],
         "signal_triggerref_e_bjet_met":["electron","muon","bJet","trigger_de_ref"],
         "signal_triggerctl_e_bjet_met":["electron","muon","bJet","trigger_de_ref","trigger_central"],
                }
        
        #candidates
        
        candidtae_events_ref=events[selection.all(*regions["signal_triggerref_e_bjet_met"])]
        candidtae_events_ctl=events[selection.all(*regions["signal_triggerctl_e_bjet_met"])]
        
      
        
            
        #selection out
        # pt m
        output['pt_m_tr'].fill(pt_m_tr = ak.firsts(candidtae_events_ref.heepmuon.pt))
        output['pt_m_tc'].fill(pt_m_tc = ak.firsts(candidtae_events_ctl.heepmuon.pt))
        
        #eta m
        output['eta_m_tr'].fill(eta_m_tr = ak.firsts(candidtae_events_ref.heepmuon.eta))
        output['eta_m_tc'].fill(eta_m_tc = ak.firsts(candidtae_events_ctl.heepmuon.eta))
        
        #Met
        output['met_tr'].fill(met_tr = candidtae_events_ref.MET.pt)
        output['met_tc'].fill(met_tc = candidtae_events_ctl.MET.pt)
        
        #pt bjet
        output['pt_bjet_tr'].fill(pt_bjet_tr = ak.firsts(candidtae_events_ref.bjet.pt))
        output['pt_bjet_tc'].fill(pt_bjet_tc = ak.firsts(candidtae_events_ctl.bjet.pt))
        
        #eta bjet
        
        output['eta_bjet_tr'].fill(eta_bjet_tr = ak.firsts(candidtae_events_ref.bjet.eta))
        output['eta_bjet_tc'].fill(eta_bjet_tc = ak.firsts(candidtae_events_ctl.bjet.eta))
        
        
        return {dataset: output}
            
    def postprocess(self, accumulator):
        return accumulator